In [8]:
from dotenv import load_dotenv
from langchain.tools import BaseTool, Tool, tool
from langgraph.graph import StateGraph, END
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage, FunctionMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt.tool_executor import ToolExecutor
from langgraph.prebuilt import ToolInvocation
from typing import TypedDict, Sequence
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools import format_tool_to_openai_function
from langchain_openai import ChatOpenAI
import re
from agent import get_agent, load_command_schemas
import json

import os

In [9]:
commands = load_command_schemas("command_schemas.json")
system_message = SystemMessage(
        content=f"""You are an agent designed to take natural language prompts, infer intent, and map that intent
        to corresponding JupyterLab commands in JSON structure, following the correct execution order.

        Below is a list of valid JupyterLab commands. Each command includes:
          - A **title** (short name)
          - A **description** (what it does)
          - **Example phrases** (possible user inputs)
          - The **expected JSON structure** to execute the command

        Reference this list when constructing valid JupyterLab command responses:
        ```json
        {json.dumps(commands, indent=2)}
        ```
        Ensure that all responses strictly follow the JSON schema definitions.
        """
    )
    # User message
human_message = HumanMessage(content="Run the cell 3 cells above this")

    # Run agent
config = {"configurable": {"thread_id": "thread-1"}}
inputs = {"messages": [system_message, human_message]}
app = get_agent()
    
    # Extract last agent message (already a Python object, no need to parse)
result = app.invoke(inputs, config)["messages"][-1] 
print("RESULT: ", result) 

LAST MESSAGE:  I'm sorry, but the JupyterLab commands do not support running a specific cell by its relative position. You can only run the currently selected cell, all cells, or restart the kernel and run all cells.
Failed to parse JSON. Returning empty object.
RESULT:  {}


In [7]:

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

# Set up LLM
llm = ChatOpenAI(api_key=api_key, model="gpt-4", temperature=0, streaming=True)

COMMANDS_FILE = "commands.txt"
config = {"configurable": {"thread_id": "thread-1"}}



In [276]:

# Memory saver for workflow
memory = MemorySaver()

class AgentState(TypedDict):
    messages: Sequence[BaseMessage]


In [277]:
# Load command schema from a JSON file
def load_command_schemas(file_path="command_schemas.json"):
    """Load the available command schemas from a JSON file."""
    with open(file_path, "r") as file:
        return json.load(file)


In [278]:
tools = []
tool_executor = ToolExecutor(tools)
functions = [format_tool_to_openai_function(t) for t in tools]
model = llm

/var/folders/3h/z8r254ts5r58qphcnd1h46xw0000gn/T/ipykernel_34024/912262656.py:2: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [279]:

def agent(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": messages + [response]}


def should_continue(state):
    last_message = state["messages"][-1]
    return "continue" if "function_call" in last_message.additional_kwargs else "end"

def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input={**json.loads(last_message.additional_kwargs["function_call"]["arguments"])}
    )

    response = tool_executor.invoke(action)  # Fix typo here, should be `tool_executor`
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {"messages": messages + [function_message]}


In [280]:
def extract_json(state):
    """Extracts the JSON command from the agent's response."""
    messages = state["messages"]
    last_message = messages[-1].content  # Get the latest response from the agent

    # Extract JSON using regex (handles multiple formats)
    match = re.search(r'```json\n(.*?)\n```', last_message, re.DOTALL)
    if match:
        extracted_json = match.group(1)  # Capture JSON inside triple backticks
    else:
        extracted_json = last_message  # Fallback if not wrapped in ```json```

    # Try to parse the extracted JSON
    try:
        parsed_json = json.loads(extracted_json)
        print(f"Extracted JSON: {parsed_json}")
    except json.JSONDecodeError:
        print("Failed to parse JSON. Returning empty object.")
        parsed_json = {}

    return {"messages": messages + [parsed_json]}

In [281]:
workflow = StateGraph(AgentState)

# Define nodes
workflow.add_node("agent", agent)
workflow.add_node("action", call_tool)
workflow.add_node("extract_json", extract_json)  # New node!

# Set entry point
workflow.set_entry_point("agent")

# Conditional edges:
workflow.add_conditional_edges("agent", should_continue, {
    "continue": "action",  # If more actions needed, call tools
    "end": "extract_json"  # If response is ready, extract JSON before finishing
})

# Ensure action results are processed again by the agent
workflow.add_edge("action", "agent")

# Extract JSON before final output
workflow.add_edge("extract_json", END)

# Compile workflow
app = workflow.compile(checkpointer=memory)


In [282]:
# Load the command schemas
commands = load_command_schemas()

# Create a structured system message to educate the agent
system_message = SystemMessage(
    content=f"""You are an agent designed to take natural language prompts, infer intent, and map that intent
    to corresponding JupyterLab commands in JSON structure, following the correct execution order.

    Below is a list of valid JupyterLab commands. Each command includes:
      - A **title** (short name)
      - A **description** (what it does)
      - **Example phrases** (possible user inputs)
      - The **expected JSON structure** to execute the command

    Reference this list when constructing valid JupyterLab command responses:
    ```json
    {json.dumps(commands, indent=2)}
    ```
    Ensure that all responses strictly follow the JSON schema definitions.
    """
)

# Example user message
human_message = HumanMessage(
    content="Run the current cell"
)

# Format inputs for agent processing
inputs = {"messages": [system_message, human_message]}

In [288]:
result = app.invoke(inputs, config)["messages"][-1]
result

Extracted JSON: {'name': 'notebook:run-cell', 'args': {'activate': True}}


{'name': 'notebook:run-cell', 'args': {'activate': True}}

{'name': 'notebook:run-cell', 'args': {'activate': True}}

In [283]:

for output in app.stream(inputs, config):
    for key, value in output.items():
        print(f"Output from node: '{key}':")
        print("---")
        print(value)
        print("\n---\n")

Output from node: 'agent':
---
{'messages': [SystemMessage(content='You are an agent designed to take natural language prompts, infer intent, and map that intent\n    to corresponding JupyterLab commands in JSON structure, following the correct execution order.\n\n    Below is a list of valid JupyterLab commands. Each command includes:\n      - A **title** (short name)\n      - A **description** (what it does)\n      - **Example phrases** (possible user inputs)\n      - The **expected JSON structure** to execute the command\n\n    Reference this list when constructing valid JupyterLab command responses:\n    ```json\n    {\n  "$schema": "https://json-schema.org/draft/2020-12/schema",\n  "commands": [\n    {\n      "title": "Run Cell",\n      "description": "Runs the currently selected cell in the Jupyter notebook.",\n      "example_phrases": [\n        "Run this cell",\n        "Execute the selected cell",\n        "Go ahead and run the active code block"\n      ],\n      "json_schema"

In [70]:
from langchain_core.messages import AIMessage

response = app.invoke(inputs, config)
if isinstance(response["messages"][-1], AIMessage):
    print("Agent thought process:", response["messages"][-1].response_metadata)


Agent thought process: {'finish_reason': 'stop', 'model_name': 'gpt-4-0613'}


In [247]:
from jsonschema import validate, ValidationError

# The schema for validating AI responses
schema = {
    "type": "object",
    "properties": {
        "name": {"const": "notebook:run-cell"},
        "args": {
            "type": "object",
            "properties": {
                "activate": {"type": "boolean"}
            },
            "required": ["activate"]
        }
    },
    "required": ["name", "args"]
}

# Example command from AI
command_response = {
    "name": "notebook:run-cell",
    "args": {"activate": True}
}

try:
    validate(instance=command_response, schema=schema)
    print("✅ Valid command! Sending to JupyterLab...")
    # Call JupyterLab execution function here
except ValidationError as e:
    print("❌ Invalid command:", e)


✅ Valid command! Sending to JupyterLab...
